In [4]:
import sympy as sy
import numpy as np
from sympy.solvers import solve

In [5]:
x1, mx1, x2, mx2 = sy.var('x1, mu_1, x2, mu_2', real=True)
a, b = sy.var('a, b', positive=True, integer=True)

In [6]:
def m_ab(x1, a, x2, b):
    """Co-moment of x1**a and x2**b""" 
    return sy.expand((x1 - mx1)**a * (x2 - mx2)**b)

In [7]:
# An version of the expression that uses stohastic vars. `E` is handled as a linear function,
# but in our case it does not help much as expectations have only one argument each.
# Also ideally r and t should be declared as sample valaues to get complete expression (top level Sum).
from sympy.stats import P, E, variance, Die, Normal
from sympy.stats.rv import RandomSymbol
mr, dr = sy.var('mu_r, dr')
mt, dt = sy.var('mu_t, dt')
r = RandomSymbol('r')
t = RandomSymbol('t')
sy.expand(E((r - E(r))**2*(t - E(t))**3)) \
    .replace(E(r), mr)  \
    .replace(E(t), mt)
# There seem to be no standard way to sort members by powers (sp.Poly does print them in a
# consistent way but that order is lost when the sum expression is extracted from it)

-mu_r**2*mu_t**3 + 3*mu_r**2*mu_t**2*t - 3*mu_r**2*mu_t*t**2 + mu_r**2*t**3 + 2*mu_r*mu_t**3*r - 6*mu_r*mu_t**2*r*t + 6*mu_r*mu_t*r*t**2 - 2*mu_r*r*t**3 - mu_t**3*r**2 + 3*mu_t**2*r**2*t - 3*mu_t*r**2*t**2 + r**2*t**3

In [8]:
# This version extracts common expressions, but in our case it does not seem to be a significant optimization.
# Need to benchmark it to see the impact.
(common_vals, exprs) =  sy.cse(sy.simplify(m_ab(x1, 2, x2, 4)))
(common_vals, exprs)

([(x0, mu_1**2),
  (x1, mu_2**4),
  (x2, x2**4),
  (x3, x1**2),
  (x4, mu_1*x1),
  (x5, 2*x4),
  (x6, mu_2*x2**3),
  (x7, 8*x4),
  (x8, mu_2**3*x2),
  (x9, 4*x0),
  (x10, 4*x3),
  (x11, mu_2**2*x2**2),
  (x12, 6*x11)],
 [x0*x1 + x0*x12 + x0*x2 + x1*x3 - x1*x5 - x10*x6 - x10*x8 - 12*x11*x4 + x12*x3 + x2*x3 - x2*x5 + x6*x7 - x6*x9 + x7*x8 - x8*x9])

In [9]:
# An example creating a callable function in sympy. It requires sympy dependency and will have to 
# have a parameter for each member (x1^a1*x2^b2).
# https://docs.sympy.org/latest/modules/utilities/lambdify.html
import inspect
lf = sy.lambdify((x1, mx1, x2, mx2), sy.simplify(m_ab(x1, 2, x2, 4)), "numpy")
print(inspect.getsource(lf))
lf(23, 21, 1.1, 1.3)

def _lambdifygenerated(x1, mu_1, x2, mu_2):
    return mu_1**2*mu_2**4 - 4*mu_1**2*mu_2**3*x2 + 6*mu_1**2*mu_2**2*x2**2 - 4*mu_1**2*mu_2*x2**3 + mu_1**2*x2**4 - 2*mu_1*mu_2**4*x1 + 8*mu_1*mu_2**3*x1*x2 - 12*mu_1*mu_2**2*x1*x2**2 + 8*mu_1*mu_2*x1*x2**3 - 2*mu_1*x1*x2**4 + mu_2**4*x1**2 - 4*mu_2**3*x1**2*x2 + 6*mu_2**2*x1**2*x2**2 - 4*mu_2*x1**2*x2**3 + x1**2*x2**4



0.006399999999757711

In [10]:
# The method that lambdify uses. Does not do any namespace subtitutions, though 
# (like adapting/converting operations to numpy or scypy).
from sympy.utilities.lambdify import lambdastr
lambdastr([x1, mx1, x2, mx2], sy.simplify(m_ab(x1, 2, x2, 4)))

'lambda x1,mu_1,x2,mu_2: (mu_1**2*mu_2**4 - 4*mu_1**2*mu_2**3*x2 + 6*mu_1**2*mu_2**2*x2**2 - 4*mu_1**2*mu_2*x2**3 + mu_1**2*x2**4 - 2*mu_1*mu_2**4*x1 + 8*mu_1*mu_2**3*x1*x2 - 12*mu_1*mu_2**2*x1*x2**2 + 8*mu_1*mu_2*x1*x2**3 - 2*mu_1*x1*x2**4 + mu_2**4*x1**2 - 4*mu_2**3*x1**2*x2 + 6*mu_2**2*x1**2*x2**2 - 4*mu_2*x1**2*x2**3 + x1**2*x2**4)'

// ScyPy matrices https://docs.sympy.org/latest/tutorials/intro-tutorial/matrices.html

In [11]:
nv = 5
na = 2
nb = 3
powers = [[0] * nb] * na
for a in range(na):
    for b in range(nb):
        powers[a][b] = np.zeros((nv, nv))
# print(powers)

In [12]:
sample = np.random.normal(1, 2, size=(33, nv)) # Samples in rows
Xt = sample.T
X = sample
for a in range(na):
    XX = np.matmul(X.T, X)
    for b in range(nb):
        assert XX.shape == (nv, nv)
        powers[a][b] = XX
    
print(XX)

[[153.18605846   2.11515729  46.84432477  -2.48313643   3.53301852]
 [  2.11515729 145.44897258  39.44217674  34.17641351   4.29193466]
 [ 46.84432477  39.44217674 233.71707164   3.81280962   3.91681438]
 [ -2.48313643  34.17641351   3.81280962  84.53409029  -6.4107536 ]
 [  3.53301852   4.29193466   3.91681438  -6.4107536  188.44432369]]


// Just keeping a note for now. Batched GEMM in cuda (is it in cupy yet?)
https://docs.nvidia.com/cuda/cublas/index.html#cublas-t-gemmbatchedm

In [13]:
import scipy.linalg.blas as blas

BLAS GEMM 
https://spec.oneapi.io/versions/latest/elements/oneMKL/source/domains/blas/gemm.html#onemkl-blas-gemm`

In [23]:
# a = blas.dgemm(1.0, a, b, 1.0, c, False, False, True)

print(blas.dgemm.__doc__)

c = dgemm(alpha,a,b,[beta,c,trans_a,trans_b,overwrite_c])

Wrapper for ``dgemm``.

Parameters
----------
alpha : input float
a : input rank-2 array('d') with bounds (lda,ka)
b : input rank-2 array('d') with bounds (ldb,kb)

Other Parameters
----------------
beta : input float, optional
    Default: 0.0
c : input rank-2 array('d') with bounds (m,n)
overwrite_c : input int, optional
    Default: 0
trans_a : input int, optional
    Default: 0
trans_b : input int, optional
    Default: 0

Returns
-------
c : rank-2 array('d') with bounds (m,n)



In [46]:
a1, a2, a3, b1, b2, b3 = sy.symbols('a1 a2 a3 b1 b2 b3')
A1 = sy.Matrix([[a1, a2, a3]])
A2 = A1.applyfunc(lambda x: x ** 2)
assert A1 != A2 # Verify if matrices are updated in-place.
A3 = A1.applyfunc(lambda x: x ** 3)
A4 = A1.applyfunc(lambda x: x ** 4)

In [39]:
A1

Matrix([[a1, a2, a3]])

In [40]:
A1.T * A1

Matrix([
[a1**2, a1*a2, a1*a3],
[a1*a2, a2**2, a2*a3],
[a1*a3, a2*a3, a3**2]])

In [41]:
A1.T * A2

Matrix([
[   a1**3, a1*a2**2, a1*a3**2],
[a1**2*a2,    a2**3, a2*a3**2],
[a1**2*a3, a2**2*a3,    a3**3]])

In [42]:
A1.T * A3

Matrix([
[   a1**4, a1*a2**3, a1*a3**3],
[a1**3*a2,    a2**4, a2*a3**3],
[a1**3*a3, a2**3*a3,    a3**4]])

In [43]:
A1.T * A4

Matrix([
[   a1**5, a1*a2**4, a1*a3**4],
[a1**4*a2,    a2**5, a2*a3**4],
[a1**4*a3, a2**4*a3,    a3**5]])

In [44]:
mo12 = m_ab(a1, 1, a2, 2)
mo12

a1*a2**2 - 2*a1*a2*mu_2 + a1*mu_2**2 - a2**2*mu_1 + 2*a2*mu_1*mu_2 - mu_1*mu_2**2

In [31]:
N = sy.ones(cols=2, rows=1)

In [32]:
K = sy.Matrix([1, a1, a1 ** 2, a2, a2 ** 2])
K * K.T

Matrix([
[    1,       a1,       a1**2,       a2,       a2**2],
[   a1,    a1**2,       a1**3,    a1*a2,    a1*a2**2],
[a1**2,    a1**3,       a1**4, a1**2*a2, a1**2*a2**2],
[   a2,    a1*a2,    a1**2*a2,    a2**2,       a2**3],
[a2**2, a1*a2**2, a1**2*a2**2,    a2**3,       a2**4]])

In [33]:
B = sy.Matrix([1, a1, a2, a1 ** 2, a2 ** 2])
ZZ = B * B.T
ZZ

Matrix([
[    1,       a1,       a2,       a1**2,       a2**2],
[   a1,    a1**2,    a1*a2,       a1**3,    a1*a2**2],
[   a2,    a1*a2,    a2**2,    a1**2*a2,       a2**3],
[a1**2,    a1**3, a1**2*a2,       a1**4, a1**2*a2**2],
[a2**2, a1*a2**2,    a2**3, a1**2*a2**2,       a2**4]])

In [34]:
ZZ.multiply_elementwise(sy.Matrix([
    [0, mx2**2, 2 * mx1 * mx2, 0, -mx1],
    [0,      0,      -2 * mx2, 0,    1],
    [0,      0,             0, 0,    0],
    [0,      0,             0, 0,    0],
    [0,      0,             0, 0,    0],
]))

Matrix([
[0, a1*mu_2**2, 2*a2*mu_1*mu_2, 0, -a2**2*mu_1],
[0,          0,  -2*a1*a2*mu_2, 0,    a1*a2**2],
[0,          0,              0, 0,           0],
[0,          0,              0, 0,           0],
[0,          0,              0, 0,           0]])

In [35]:
print(sy.expand((x2 - mx2)**4))

mu_2**4 - 4*mu_2**3*x2 + 6*mu_2**2*x2**2 - 4*mu_2*x2**3 + x2**4


In [36]:
mu_2**4 - 4*mu_2**3*x2 + 6*mu_2**2*x2**2 - 4*mu_2*x2**3 + x2**4

mu_2**4 - 4*mu_2**3*x2 + 6*mu_2**2*x2**2 - 4*mu_2*x2**3 + x2**4